In [1]:
from os import listdir
import cv2
from typing import List


### Function to get label by video id

In [2]:
import sys

sys.path.insert(0, "../utils")

from data_reader import get_label_from_id


### CONFIG

In [3]:
n_imgs = 3000

### Input folder and img names

In [4]:
input_folder = "../../data/WLASL2000_frames"
input_img_names = listdir(input_folder)

# Cut off the first n_imgs images
input_img_names = input_img_names[:n_imgs]

input_img_names[:5]

['00295.png', '00333.png', '00335.png', '00336.png', '00337.png']

#### Randomize the order of the images

In [5]:
import random

random.shuffle(input_img_names)

input_img_names[:5]

['05603.png', '09924.png', '07477.png', '04764.png', '08345.png']

### Divide input imgs in train, validation and test

In [6]:
input_len = len(input_img_names)
(train, test, validation) = [int(perc * input_len) for perc in [0.7, 0.2, 0.1]]

train_imgs = input_img_names[:train]
test_imgs = input_img_names[train : train + test]
validation_imgs = input_img_names[train + test :]


### Get labels for train, test and validations imgs

In [9]:
def get_labels_from_set(set: List[str]) -> List[str]:
    labels = [get_label_from_id(img.split(".")[0]) for img in set]
    return labels


train_labels = get_labels_from_set(train_imgs)
test_labels = get_labels_from_set(test_imgs)
validation_labels = get_labels_from_set(validation_imgs)


['because',
 'champion',
 'bracelet',
 'bake',
 'bush',
 'baby',
 'cabbage',
 'alphabet',
 'card',
 'basketball',
 'basement',
 'before',
 'cake',
 'candy',
 'bedroom',
 'benefit',
 'around',
 'barely',
 'accomplish',
 'beginning',
 'balance',
 'alone',
 'assist',
 'arizona',
 'area',
 'bee',
 'bark',
 'bowling',
 'bet',
 'apartment',
 'breakdown',
 'blame',
 'above',
 'awkward',
 'art',
 'article',
 'always',
 'after',
 'both',
 'cent',
 'ahead',
 'book',
 'between',
 'ask',
 'awkward',
 'book',
 'arrest',
 'black',
 'breakdown',
 'babysitter',
 'can',
 'appear',
 'attorney',
 'baby',
 'after',
 'bookstore',
 'car',
 'best',
 'average',
 'age',
 'aunt',
 'certificate',
 'accept',
 'big',
 'art',
 'cat',
 'apple',
 'arrest',
 'camp',
 'brown',
 'ago',
 'below',
 'author',
 'bracelet',
 'adverb',
 'again',
 'below',
 'bread',
 'background',
 'before',
 'ball',
 'battery',
 'bracelet',
 'button',
 'book',
 'again',
 'buy',
 'answer',
 'background',
 'also',
 'attorney',
 'book',
 'bull',

### Entrenar red de clasificación

### Mostrar imagen

In [8]:
image = cv2.imread(f"{input_folder}/{input_img_names[1]}")
cv2.imshow("image", image)
cv2.waitKey(0)


-1